## Purpose

This notebook is designed to rank the parameter sets from the LHS using data genereated from single and double accession simulations from the pollen competition ABM. The ranking is performed as follows. 

1. Calculate the absolute difference between the mean pollen tube length from each single-accession model simulation with the mean estimated from the Kolba-Capaldi paper at 3, 6, 9, and 24 hours post-germination.
2. Conduct a Kolmogorov-Smirnov (KS) test to determine if the pollen tube lengths from each single-accession model simulation differ from an exponential distribution (with mean from the Kolba-Capaldi paper) at 3, 6, 9, and 24 hours post-germination.
3. Calculate the absolute difference between the proportion of seeds fertilized from each double-accession model simulation with the empirically known proportion of seeds fertilized. 
4. Rank the parameter sets according to the following methods. 
    * **Mean Rank**: Total sum (across all realizations) of the values generated from step 1. 
    * **Distribution Rank**: Total count (across all realizations) of hypothesis tests ran in step 2 that fail to reject the null hypothesis (that the distributions are the same) at the 0.05 significance level. 
    * **Fertilized Ovules Rank**: Total sum (across all realizations) of the values generated from step 3.
5. Aggregate the ranks from step 4 for each parameter according to two methods. 
    * **Mean Length \& Fertilized Ovules (MLFO)**: The sum of the parameter's Mean Rank and Fertilized Ovules Rank
    * **Distribution \& Fertilized Ovules (DFO)**: The sum of the parameter's Distribution Rank and Fertilized Ovules Rank

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import pickle
import ast
import os

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
# Read in the model generated data for the single accession. 
cols = ['accession', 'count patches with [pcolor = yellow]','count patches with [pcolor = blue] / 6', 'count patches with [pcolor = red] / 6', \
     'r3L', 'r6L', 'r9L', 'r24L','LHS-paramcombo-counter', 'r3C', 'r6C', 'r9C', 'r24C']

single_acc_df = pd.read_csv(os.getcwd() + '\\Experiment Data\\calibration_data_single_accession', usecols=cols, skiprows=6, sep=',')

# Modify column names 
single_acc_df = single_acc_df.rename(columns={'count patches with [pcolor = yellow]': 'num_unfert_ovules',\
     'count patches with [pcolor = blue] / 6': 'num_col_ovules', 'count patches with [pcolor = red] / 6': 'num_ler_ovules',\
     'r3L': 'hr3_ler_tube_lengths', 'r6L': 'hr6_ler_tube_lengths', 'r9L': 'hr9_ler_tube_lengths', 'r24L': 'hr24_ler_tube_lengths',\
     'LHS-paramcombo-counter': 'param_num', 'r3C': 'hr3_col_tube_lengths', 'r6C': 'hr6_col_tube_lengths', 'r9C': 'hr9_col_tube_lengths',\
        'r24C': 'hr24_col_tube_lengths'})

In [ ]:
# Read in the model generated data for the double accession
cols = ['accession', 'count patches with [pcolor = yellow]','count patches with [pcolor = blue] / 6', 'count patches with [pcolor = red] / 6', \
     'r3L', 'r6L', 'r9L', 'r24L','LHS-paramcombo-counter', 'r3C', 'r6C', 'r9C', 'r24C']
double_acc_df = pd.read_csv(os.getcwd() + '\\Experiment Data\\calibration_data_double_accession', usecols=cols, skiprows=6, sep=',')

# Modify column names 
double_acc_df = double_acc_df.rename(columns={'count patches with [pcolor = yellow]': 'num_unfert_ovules',\
     'count patches with [pcolor = blue] / 6': 'num_col_ovules', 'count patches with [pcolor = red] / 6': 'num_ler_ovules',\
     'r3L': 'hr3_ler_tube_lengths', 'r6L': 'hr6_ler_tube_lengths', 'r9L': 'hr9_ler_tube_lengths', 'r24L': 'hr24_ler_tube_lengths',\
     'LHS-paramcombo-counter': 'param_num', 'r3C': 'hr3_col_tube_lengths', 'r6C': 'hr6_col_tube_lengths', 'r9C': 'hr9_col_tube_lengths',\
        'r24C': 'hr24_col_tube_lengths'})

In [ ]:
# Store the mean pollen tube lengths as stated in the Kolba-Capaldi paper in a dictionary. All pollen tube means are 
# based on single accession data and are measured at 3, 6, 9, and 24 hours post-germination. 
known_tube_means = {}
known_tube_means['ler-only3'] = 0.066; known_tube_means['ler-only6'] = 0.094; known_tube_means['ler-only9'] = 0.251; known_tube_means['ler-only24'] = 0.324
known_tube_means['col-only3'] = 0.093; known_tube_means['col-only6'] = 0.144; known_tube_means['col-only9'] = 0.342; known_tube_means['col-only24'] = 0.375

# Create a list of exponential CDFs with shape parameters as estimated by Kolba-Capaldi paper. 
KC_ler_cdfs = []
KC_col_cdfs = []

for time in [3, 6, 9, 24]:
    KC_ler_cdfs.append(stats.expon(loc = 0, scale = known_tube_means['ler-only'+str(time)]).cdf)
    KC_col_cdfs.append(stats.expon(loc = 0, scale = known_tube_means['col-only'+str(time)]).cdf)

In [ ]:
def tube_clean(tubes):
    
    # Inputs
    # tubes - string object of space separated pollen tube lengths. 
    
    # Outputs:
    # tubes - numpy array of the input without zeros. 

    tubes = tubes.replace(" ",",")
    tubes = np.array(ast.literal_eval(tubes))
    
    if tubes.any() != 0:
        new_tubes = tubes[np.nonzero(tubes)]
    else:
        new_tubes = np.array([0])
    
    return new_tubes

In [ ]:
def sim_score(accession, col3, ler3, col6, ler6, col9, ler9, col24, ler24, known_tube_means, KC_ler_cdfs, KC_col_cdfs):
    
    # Inputs: 
    # colx / lerx - object with pollen tube lengths for col/ler at time x for one parameter realization. 
    
    # Outputs:
    # 1. sum (across all time values) of absolute difference between model mean pollen tube length and the 
    #    predicted value according to Kolba-Capaldi paper. 
    # 2. count (across all time values) where the KS test fails to reject the null hypothesis that the distribution of 
    #    pollen tube lengths from the model is the same as the Kolba-Capaldi paper.
    
    # Calculate the output described in step 1 (the first if statement detects which accession data to use, then 
    # we clean up the NetLogo ouput and perform this calculation in step 1, but if there are less than 20
    # pollen tubes with nonzero length we just return a large quantity.)
    if accession == 'ler-only':
        clean3, clean6, clean9, clean24 = tube_clean(ler3), tube_clean(ler6), tube_clean(ler9), tube_clean(ler24)
    else:
        clean3, clean6, clean9, clean24 = tube_clean(col3), tube_clean(col6), tube_clean(col9), tube_clean(col24)
    
    if min(len(clean3), len(clean6), len(clean9), len(clean24)) < 20:
        abs_diff_mean = 999999
    else:
        abs_diff_mean = abs(np.mean(clean3) - known_tube_means[accession + '3']) + \
        abs(np.mean(clean6) - known_tube_means[accession + '6']) + \
        abs(np.mean(clean9) - known_tube_means[accession + '9']) + \
        abs(np.mean(clean24) - known_tube_means[accession + '24'])
    
    if accession == 'ler-only':
        if len(clean3) > 19:
            pval3 = stats.kstest(clean3, KC_ler_cdfs[0])[1]
        else:
            pval3 = 0
            
        if len(clean6) > 19:
            pval6 = stats.kstest(clean6, KC_ler_cdfs[1])[1]
        else:
            pval6 = 0
            
        if len(clean9) > 19:
            pval9 = stats.kstest(clean9, KC_ler_cdfs[2])[1]
        else:
            pval9 = 0
            
        if len(clean24) > 19:
            pval24 = stats.kstest(clean24, KC_ler_cdfs[3])[1]
        else:
            pval24 = 0
    else:
        if len(clean3) > 19:
            pval3 = stats.kstest(clean3, KC_col_cdfs[0])[1]
        else:
            pval3 = 0
            
        if len(clean6) > 19:
            pval6 = stats.kstest(clean6, KC_col_cdfs[1])[1]
        else:
            pval6 = 0
            
        if len(clean9) > 19:
            pval9 = stats.kstest(clean9, KC_col_cdfs[2])[1]
        else:
            pval9 = 0
            
        if len(clean24) > 19:
            pval24 = stats.kstest(clean24, KC_col_cdfs[3])[1]
        else:
            pval24 = 0
    
    KS_count = 1*(pval3 > 0.05) + 1*(pval6 > 0.05) + 1*(pval9 > 0.05) + 1*(pval24 > 0.05)
    
    return abs_diff_mean, KS_count

In [ ]:
# For each realization of the single accession model determine a mean score and distribution (KS) score. 
for k in range(len(single_acc_df)):
    
    if k % 10000 == 0:
        print(100*k/len(single_acc_df))
    sim = single_acc_df.iloc[k]
    
    val1, val2 = sim_score(sim.accession,
                  sim.hr3_col_tube_lengths, sim.hr3_ler_tube_lengths,
                  sim.hr6_col_tube_lengths, sim.hr6_ler_tube_lengths,
                  sim.hr9_col_tube_lengths, sim.hr9_ler_tube_lengths,
                  sim.hr24_col_tube_lengths, sim.hr24_ler_tube_lengths,
                  known_tube_means, KC_ler_cdfs, KC_col_cdfs)
    single_acc_df.at[k,'mean_score'] = val1
    single_acc_df.at[k, 'ks_score'] =  val2

In [ ]:
# For each realization of the double accession model determine a fertilized ovule score. 
double_acc_df['fert_ovule_score'] = abs((60 - double_acc_df['num_unfert_ovules'])/60 - 0.93012)

In [ ]:
# Create a data frame that holds the results grouped and summed by parameter number for the single accession scores
single_acc_scores_df = single_acc_df[['param_num', 'mean_score', 'ks_score']].groupby('param_num').sum()

In [ ]:
# Create a data frame that holds the results grouped and summed by parameter number for the double accession scores
double_acc_scores_df = double_acc_df[['param_num', 'fert_ovule_score']].groupby('param_num').sum()

In [ ]:
# Determine the ranks.
MLFO_ranks = single_acc_scores_df['mean_score'].rank(ascending=True, method='max') + double_acc_scores_df['fert_ovule_score'].rank(ascending=True, method='max')
DFO_ranks = single_acc_scores_df['ks_score'].rank(ascending=False, method='max') + double_acc_scores_df['fert_ovule_score'].rank(ascending=True, method='max')

In [ ]:
MLFO_ranks.sort_values(ascending=True).head(100)

In [ ]:
DFO_ranks.sort_values(ascending=True).head(100)

In [ ]:
# Pickle the rankings and modified dfs. 
pickle.dump(MLFO_ranks, open("MLFO_ranks.pickle","wb"))
pickle.dump(DFO_ranks, open("DFO_ranks.pickle","wb"))

In [ ]:
# Read in the LHS, print off the best parameters. 
LHS_df = pd.read_csv(os.getcwd() + '\\Experiment Data\\LHS.csv', sep=',', header=None)

In [ ]:
LHS_df.loc[MLFO_ranks.sort_values(ascending=True).head(100).index.tolist()].to_csv('best_mean_params.csv', index=False, header=None)
LHS_df.loc[DFO_ranks.sort_values(ascending=True).head(100).index.tolist()].to_csv('best_distr_params.csv', index=False, header=None)